In [5]:
# import arcpy
# from arcpy import env
import os
import numpy as np
import keyring
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
# import fiona # used to import gdbs 
import pandas as pd
import glob

# arcpy.env.overwriteOutput = True
# arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [6]:
# Sign into ArcGIS Online
un = 'analytics_wfrc'
pw = keyring.get_password('Analytics AGOL', un)
gis = GIS(username=un, password=pw)

In [7]:
pd.read_pickle(r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq196\Progression_Metrics\run_196_year_2019_parcel_progression_metrics.pkl").columns

Index(['parcel_id', 'county_id', 'zone_id', 'parcel_acres', 'land_value',
       'max_far', 'max_dua', 'has_buildings', 'was_developed',
       'was_redeveloped', 'developable', 'residential_units', 'job_spaces',
       'building_sqft', 'non_residential_sqft', 'residential_sqft',
       'unit_price_non_residential', 'res_price_per_sqft', 'is_sf', 'is_mf',
       'is_industrial', 'is_retail', 'is_office', 'is_govt', 'is_mixeduse',
       'is_other', 'year_built', 'building_count', 'jobs_accom_food',
       'jobs_gov_edu', 'jobs_health', 'jobs_manuf', 'jobs_office',
       'jobs_other', 'jobs_retail', 'jobs_wholesale', 'households_count',
       'hhpop', 'non_res_value', 'res_value', 'total_value',
       'job_spaces_added', 'jobs_accom_food_added', 'jobs_gov_edu_added',
       'jobs_health_added', 'jobs_manuf_added', 'jobs_office_added',
       'jobs_other_added', 'jobs_retail_added', 'jobs_wholesale_added',
       'res_units_added', 'households_added', 'acreage_dev', 'acreage_redev',
 

In [8]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [9]:
base_year = 2019
# base_year = 2015

In [10]:
if base_year == 2015:
    outputs_pkl = r'.\\Outputs\progession_metrics_2015_pkl'
    outputs_csv = r'.\\Outputs\progession_metrics_2015_csv'
if base_year == 2019:
    outputs_pkl = r'.\\Outputs\progession_metrics_pkl'
    outputs_csv = r'.\\Outputs\progession_metrics_csv'

for d in [outputs_pkl, outputs_csv]:
    if not os.path.exists(d):
        os.makedirs(d)

In [11]:
# Parcel Equivalency Table
if base_year == 2015:
    eq = pd.read_csv(r".\Inputs\parcel_eq_2015_v1.csv")
if base_year == 2019:
    eq = pd.read_csv(r".\Inputs\parcel_eq_v7.csv")
    
# centers_eq_ids = eq[eq['CENTER_NAME'].isna() == False]['parcel_id'].to_list()

# # centers shape
# centers_sdf = pd.DataFrame.spatial.from_featureclass(r".\Inputs\WC_2050_Centers.shp")

In [12]:
if base_year == 2015:
    remm_folder_1 = r"\\server1\Volumef\SHARED\Josh\REMM Runs\Progression_Metrics_2015_1"
    remm_folder_2 = r"\\server1\Volumef\SHARED\Josh\REMM Runs\Progression_Metrics_2015_2"
    remm_folder_3 = r"\\server1\Volumef\SHARED\Josh\REMM Runs\Progression_Metrics_2015_3"
    remm_folder_4 = r"\\server1\Volumef\SHARED\Josh\REMM Runs\Progression_Metrics_2015_4"
    remm_folder_5 = r"\\server1\Volumef\SHARED\Josh\REMM Runs\Progression_Metrics_2015_5"
    remm_folder_6 = r"\\server1\Volumef\SHARED\Josh\REMM Runs\Progression_Metrics_2015_6"

if base_year == 2019:
    remm_folder_1 = r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\andy231\Progression_Metrics"
    remm_folder_2 = r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq196\Progression_Metrics"
    remm_folder_3 = r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq238\Progression_Metrics"
    remm_folder_4 = r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq273\Progression_Metrics"
    remm_folder_5 = r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq339\Progression_Metrics"
    remm_folder_6 = r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq376\Progression_Metrics"

In [13]:
remm_progression_folders = [remm_folder_1, remm_folder_2, remm_folder_3, remm_folder_4, remm_folder_5, remm_folder_6]

In [14]:
# # csv version
# def get_table_ignore_base(path, year):
#     csvs = glob.glob(os.path.join(path, f'run_*_year_{year}_parcel_progression_metrics.csv'))
#     csvs = [csv for csv in csvs if 'base'not in csv]
#     if len(csvs) > 1:
#         print('warning multiple tables were globbed; only the first will be returned')
#     return pd.read_csv(csvs[0])

In [15]:
# pkl version
def get_table_ignore_base(path, year):
    pkls = glob.glob(os.path.join(path, f'run_*_year_{year}_parcel_progression_metrics.pkl'))
    pkls = [pkl for pkl in pkls if 'base'not in pkl]
    if len(pkls) > 1:
        pkls.sort(reverse=True)
        print('Warning: multiple tables were globbed; only the first will be returned')
    return pd.read_pickle(pkls[0])

In [16]:
def prepare_df(df, year):
    df = df.set_index('parcel_id')
    df.loc[(df['is_sf']==1), 'sf_units'] = df['residential_units']
    df.loc[(df['is_mf']==1), 'mf_units'] = df['residential_units']
    df['industrial_jobs'] = df['jobs_wholesale'] + df['jobs_manuf']
    df['retail_jobs'] = df['jobs_retail'] + df['jobs_accom_food']
    df['office_jobs'] = df['jobs_office'] + df['jobs_gov_edu'] + df['jobs_health'] + df['jobs_other']
    df.loc[(df['has_buildings'] != 1), 'vacant_acres'] = df['parcel_acres']
    df.loc[(df['has_buildings'] != 1) & (df['developable'] == 1), 'vacant_devacres'] = df['parcel_acres']
    df['vacant_acres'].fillna(0, inplace=True)
    df['vacant_devacres'].fillna(0, inplace=True)
    df['households'] = df['households_count']

    # df = df[['sf_units', 'mf_units', 'households', 'hhpop', 
    #          'job_spaces', 'industrial_jobs', 'retail_jobs', 'office_jobs', 
    #          'vacant_acres', 'vacant_devacres', 'non_res_value', 'res_value', 'total_value', 'non_residential_sqft']].copy()
    
    df = df[['sf_units', 'mf_units', 'households', 'hhpop', 
             'industrial_jobs', 'retail_jobs', 'office_jobs',
             'job_spaces', 'jobs_accom_food',
             'jobs_gov_edu', 'jobs_health', 'jobs_manuf', 'jobs_office',
             'jobs_other', 'jobs_retail', 'jobs_wholesale', 
             'vacant_acres', 'vacant_devacres', 'non_res_value', 'res_value', 'total_value', 'non_residential_sqft']].copy()

    return df.fillna(0)

In [17]:
if base_year == 2019:
    # statewide_se  = gis.content.get('8577570d0df6451f90bbd52b2bd3461c').tables[0].query(where='1=1').sdf # ~4-min load
    wf_se = statewide_se[statewide_se['SUBAREAID'] == 1].copy()
    wf_se = wf_se[['YEAR','SA_TAZID', 'TOTHH', 'HHPOP', 'RETEMP','INDEMP','OTHEMP','TYPEMP', 'RETL', 'FOOD', 'MANU', 'WSLE', 'OFFI', 'GVED', 'HLTH', 'OTHR']].copy() # retail, industrial, office, typical
    wf_se.rename({'SA_TAZID':'TAZID_900'}, axis=1, inplace=True)

NameError: name 'statewide_se' is not defined

In [ ]:
# base = centers_sdf[['CenterName', 'DEVACRES', 'SHAPE']].copy()
for year in range(base_year,2051):
    
    if year % 3 == 0:
        print(year)

    dfs_current_year = [get_table_ignore_base(f, year) for f in remm_progression_folders]
    dfs_processed = [prepare_df(df, year) for df in dfs_current_year] 

    # stack average the 6 runs together
    data_stack = pd.concat(dfs_processed)

    # calculate median
    median_vars =  ['job_spaces' , 'vacant_acres', 'vacant_devacres', 'non_res_value', 'res_value', 'total_value', 'non_residential_sqft']
    median = data_stack.groupby(data_stack.index).median().reset_index()
    median = median[median_vars]

    # calculate average
    average = data_stack.groupby(data_stack.index).mean().reset_index()
    average.drop(median_vars, axis=1, inplace=True)
    
    
    # determine taz id field
    if base_year == 2015:
        taz_id_field = 'TAZID_832'
    if base_year == 2019:
        taz_id_field = 'TAZID_900'
    
    # join taz id to the average table
    average = average.merge(eq[['parcel_id', 'TAZID_900']], on='parcel_id', how='left')
    wf_se_year = wf_se[wf_se['YEAR'] == year].copy()
    
    # join se data to the average table
    average = average.merge(wf_se_year, on='TAZID_900', how='left')
    
    # sum the parcel se variables by tazid
    average_taz_sum = average.groupby('TAZID_900')[['households', 'hhpop', 
             'industrial_jobs', 'retail_jobs', 'office_jobs', 'jobs_accom_food',
             'jobs_gov_edu', 'jobs_health', 'jobs_manuf', 'jobs_office',
             'jobs_other', 'jobs_retail', 'jobs_wholesale']].sum()
    
    # rename the columns
    average_taz_sum.columns = [x + '_sum' for x in average_taz_sum.columns]
    average_taz_sum = average_taz_sum.reset_index()

    # add taz sum and median variables to the average table
    average = average.merge(average_taz_sum, on='TAZID_900', how='left')
    average = average.merge(median, left_index=True, right_index=True, how='left')

    # adjust se variables using official taz forcast
    average['households'] = (average['households'] / average['households_sum']) * average['TOTHH'] 
    average['hhpop'] = (average['hhpop'] / average['hhpop_sum']) * average['HHPOP'] 
    average['industrial_jobs'] = (average['industrial_jobs'] / average['industrial_jobs_sum']) * average['INDEMP'] 
    average['retail_jobs'] = (average['retail_jobs'] / average['retail_jobs_sum']) * average['RETEMP'] 
    average['office_jobs'] = (average['office_jobs'] / average['office_jobs_sum']) * average['OTHEMP'] 
    average['jobs_accom_food'] = (average['jobs_accom_food'] / average['jobs_accom_food_sum']) * average['FOOD'] 
    average['jobs_gov_edu'] = (average['jobs_gov_edu'] / average['jobs_gov_edu_sum']) * average['GVED'] 
    average['jobs_health'] = (average['jobs_health'] / average['jobs_health_sum']) * average['HLTH'] 
    average['jobs_manuf'] = (average['jobs_manuf'] / average['jobs_manuf_sum']) * average['MANU'] 
    average['jobs_office'] = (average['jobs_office'] / average['jobs_office_sum']) * average['OFFI'] 
    average['jobs_other'] = (average['jobs_other'] / average['jobs_other_sum']) * average['OTHR'] 
    average['jobs_retail'] = (average['jobs_retail'] / average['jobs_retail_sum']) * average['RETL'] 
    average['jobs_wholesale'] = (average['jobs_wholesale'] / average['jobs_wholesale_sum']) * average['WSLE'] 

    # recalcualte total jobs and res units
    average['total_jobs'] = average['office_jobs'] + average['retail_jobs'] + average['industrial_jobs']
    average['residential_units'] = average['sf_units'] + average['mf_units']

    # fill NAs
    average = average.fillna(0)
    average = average[['parcel_id','sf_units','mf_units','households','hhpop',
                       'industrial_jobs','retail_jobs','office_jobs','job_spaces','jobs_accom_food',
                       'jobs_gov_edu','jobs_health','jobs_manuf','jobs_office','jobs_other',
                       'jobs_retail','jobs_wholesale','vacant_acres','vacant_devacres','non_res_value',
                       'res_value','total_value','non_residential_sqft','residential_units','total_jobs']]

    # export
    if base_year == 2015:
        average.to_pickle(os.path.join(outputs_pkl, f'parcel_se_b2015_{year}.pkl'))
        average.to_csv(os.path.join(outputs_csv, f'parcel_se_b2015_{year}.csv'), index=False)
    if base_year == 2019:
        average.to_pickle(os.path.join(outputs_pkl, f'parcel_se_{year}.pkl'))
        average.to_csv(os.path.join(outputs_csv, f'parcel_se_{year}.csv'), index=False)

2019
2022
2025
2028
2031
2034
2037
2040
2043
2046
2049


In [ ]:
# # test
# unpickled_df = pd.read_pickle(r"\\server1\Volumef\SHARED\Josh\2023-Official-Forecast-Files\mq376\Progression_Metrics\run_376_year_2019_parcel_progression_metrics.pkl")
# unpickled_df.columns